In [52]:
'''
Karuna Gujar
CSCI 6350-001 
Project #5
Due: 03/23/20

Description: takes as input a file containing nouns or verbs having multiple (WordNet) senses, 
and determine which of two senses are used in accompanying sentences.
'''

import numpy as np
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.wsd import lesk
from time import time
import gensim
from gensim.models import Word2Vec
from nltk.corpus import brown
from nltk import pos_tag
from nltk import word_tokenize
import json
from stanfordcorenlp import StanfordCoreNLP
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.data import find
from nltk.stem import WordNetLemmatizer

from nltk.util import ngrams
from statistics import mean 
from operator import is_not
from functools import partial
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet_ic as wic  # import that allows loading of information content
ic = wic.ic('ic-brown.dat')  


#Read the input file and extract the word, POS, sesnses and the sentences 
#and save it in a dictionary
fileName = 'sentences.txt'
sentenceSense = {}
sentences = []
f = open(fileName, 'r', encoding='utf8')
lines = f.readlines()
i_d=0
i = 0;
for line in lines:
    word = ''
    pos = ''
    sense1 = ''
    sense2 = ''
    if line != '':
        words = line.split()
        firstWord = words[0]
        if not firstWord[0].isupper():    
            i_d = i_d + 1            
            sentences = []
            word = words[0]
            pos = words[1]
            sense1 = words[2]
            sense2 = words [3]
            sentenceSense[i_d] = {'word': word, 'pos': pos, 'sense1': sense1, 'sense2':sense2, 
                            'sentences': sentences}
            if(i == len(lines)-1):
                sentenceSense[i_d] = {'word': word, 'pos': pos, 'sense1': sense1, 'sense2':sense2, 
                                'sentences': sentences}
        else:
            sentences = sentenceSense[i_d]['sentences']
            sentences.append(line)
            sentenceSense[i_d]['sentences'] = sentences             
        
    i = i + 1 
    

In [53]:
stopWords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

#Get tokens of the given sentence
def get_tokens( sentence):
    return [lemmatizer.lemmatize(x) for x in [y for y in word_tokenize(sentence) if y not in stopWords]]

#Get unigrams, bigrams and trigrams for a given snetence
def get_unibitri( sentence):
    s_ngrams = []

    token = get_tokens( sentence)
    s_ngrams.extend(list(ngrams(token, 1)))
    s_ngrams.extend(list(ngrams(token, 2)))
    s_ngrams.extend(list(ngrams(token, 3)))
    return s_ngrams

#Get the uni, bi, tri grams of the definitions of hyponyms for the given sense
'''
def get_hyponym_defs( cur_synset):
    s_ngrams = []
    for item in wn.synset(cur_synset).hyponyms():
        s_ngrams.extend(get_unibitri(item.definition()))
    return s_ngrams

#Get the uni, bi, tri grams of the definitions of hypernyms for the given sense
#Note: I am not using this method in my current approach but kept it for documentation purpose
def get_hypernym_defs( cur_synset):
    s_ngrams = []
    for item in wn.synset(cur_synset).hypernyms():
        s_ngrams.extend(get_unibitri(item.definition()))
    return s_ngrams

#Matches sentence and definition ignoring supplied words(for instance stopwaords)
#Note: I am not using this method in my current approach but kept it for documentation purpose
def matching_ngrams( source, target,ignore=[]):
    matches = []
    for item in source:
        for item1 in target:
            if item==item1:
                matches.append(item)
    ignore_tuples = [tuple([x]) for x in ignore]
    matches = [x for x in matches if x not in ignore_tuples]
            
    return matches
'''

#Gets the sense for every word in the sentence and calculates the similarity 
#score with the target sense. Then, the max score is returned. 
#the method can take an argument to provide the flexibiltiy of returning the average, max average or average max.
def get_similarity(target, sentence, score="max"):

    s_tokens = get_tokens( sentence)
    t_tok = get_tokens( target.split(".")[0])
    s_x=[]
    for token in s_tokens:
        if stemmer.stem(token)==stemmer.stem(t_tok[0]):
            continue
        s_x.append(list(filter(None.__ne__,[wn.synset(target).path_similarity(y) for y in wn.synsets(token)])))
    if score =="max":
        return max([max(x) for x in s_x if len(x) > 0])
    if score =="avg":
        return mean([mean(x) for x in s_x if len(x) > 0])
    if score =="max_avg":
        return mean([max(x) for x in s_x if len(x) > 0])
    if score =="avg_max":
        return max([mean(x) for x in s_x if len(x) > 0])

            

sentence_sense=[]            
for item in sentenceSense.items():
    '''sense1_hypo = get_hyponym_defs(item[1]["sense1"])
    sense2_hypo = get_hyponym_defs(item[1]["sense2"])
    
    sense1_hyper = get_hypernym_defs(item[1]["sense1"])
    sense2_hyper = get_hypernym_defs(item[1]["sense2"])
    
    match_1=0
    match_2=0'''

    score = "avg_max"
    for sentence in item[1]["sentences"]:
        #ignore_list=[]
        #ignore_list.append(item[1]["word"])

        #print(sentence)
        #s_ngrams=get_unibitri(sentence)
        #print(matching_ngrams(s_ngrams,sense1_hypo,ignore_list))
        
        a, b = get_similarity(item[1]["sense1"],sentence,score), get_similarity(item[1]["sense2"],sentence,score)
        
        #if the score for both the provided senses is same, NLTK lesk algorithm is used.
        if a==b:
            cur_answer = lesk(sentence, item[1]["word"] ,synsets=[wn.synset(item[1]["sense1"]),wn.synset(item[1]["sense2"])])
            sentence_sense.append((sentence, cur_answer.name()))
        else:
        
            if a > b: sentence_sense.append((sentence,item[1]["sense1"]))
            else: sentence_sense.append((sentence,item[1]["sense2"]))

#print(sentence_sense)
for s in sentence_sense:
    print(s[0].strip(),'    ', s[1])
        

Most bats eat mainly insects and bugs, but others also feed on fruits.      bat.n.01
The ball flew off the bat with a loud crack, and everyone knew it was a home run.      bat.n.02
The sound emitted by the bat has an extremely high pitch, beyong the range of human hearing.      bat.n.01
One of the weapons used in the crime was a bat.      bat.n.02
Grandma, smoking her pipe, turned around and handed the child a glass of water.      pipe.n.02
Some day he was going to pipe water into the barn making it easier to smoke jerky.      pipe.n.02
The frozen pipes burst overnight, causing a leak that badly damaged the kitchen.      pipe.n.02
As he looked at the water, Jonas filled his pipe and lit it, blowing short puffs into the air.      pipe.n.02
Next year she would graduate and would be moving away to college, far from the place she loved.      move.v.04
Why would a wealthy man move to the country and become a veterinarian?      move.v.04
Gina's eulogy was simple but moving, and left everyone

A detailed explanation of the scoring method you chose:
o How did you arrive at your final scoring method?
    I tried different approaches. I started with making a word gloss with the synonyms, hypernyms, hyponyms, meronyms, holonyms of the given word, the tokens of the definition and example for the given sense. Then used this gloss to get an overlap score with the given sentence. But this approach did not work. Every time I was getting the first sense in the list as the output.
    Another approach: I used hypernym and hyponym definitions of the target word sense to compute similarity with the given sentences. The similarity was computed using matching unigrams, bigrams and trigrams. Unfortunately I could only find matches for “bat” first example.
    Next approach I tried worked better with almost 70% (8 out of 12) accuracy by using path similarity. I am finding the path similarity score between the sense for every stemmed token in the sentence and the target sense and getting the maximum of those scores (for each sentence). Performing this with both the given senses as target. Later whichever has maximum score is the answer. If the score comes to be the same for both targets, I am using NLTK Lesk to get to the answer.


o Why did you choose the features you did?
 I did a lot of experimentation and trial and error approach to get to these features. 
In every approach I experimented, the goal was extracting all the senses for all the words in the given sentence and find ones related to the target word sense.

o Why do you think the similarity measures you chose were the best for the task?
I am using path_similarty from NLTK. This method returns a score denoting how similar two word senses are, based on the shortest path that connects the senses in the is-a (hypernym/hypnoym) taxonomy. 


o Were WordNet features or vector embedding features more important to your method? Why? 
WordNet superficially resembles a thesaurus, in that it groups words together based on their meanings. However it interlinks not just word forms—strings of letters—but specific senses of words. Hence WordNet features are more important.

An analysis of expected performance against unseen sentences
o What input may cause it to excel, and when it may not perform well? o Address this for both nouns and verbs.
The approach won’t work if the similarity score is same. If the similarity score is same, I am using NLTK Lesk to find the answer. But Lesk performance is not guaranteed. The similarity is scores for senses will be same if the sentence contains word that could be associated with any of the given senses. For example: As he looked at the water, Jonas filled his pipe and lit it, blowing short puffs into the air. The words “water” and “filled” lead to exactly same scores for both pipe.n.01 and pipe.n.02 senses.


